### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data"
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

today = date.today()
today

datetime.date(2025, 1, 17)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-17
yesterday: 2025-01-16


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-16'
ORDER BY a.name



(20, 1)

In [11]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-16' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [14]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-14,2025-01-16,5.05,5.10,23196304,5.15,5.05,B,SET,0.05,0.99
1,AIMIRT,2025-01-10,2025-01-16,10.60,10.50,1276096,10.60,10.40,B,SET,-0.10,-0.94
2,CPNREIT,2025-01-16,2025-01-16,12.20,12.10,19887990,12.30,11.80,B,SET,-0.10,-0.82
3,DIF,2025-01-16,2025-01-16,8.20,8.25,43884384,8.45,8.20,B,SET,0.05,0.61
4,GVREIT,2025-01-10,2025-01-16,6.55,6.65,6604800,6.65,6.45,B,SET,0.10,1.53
5,JMART,2025-01-16,2025-01-16,11.50,11.20,31715008,11.70,11.10,B,SET50,-0.30,-2.61
6,KCE,2025-01-15,2025-01-16,22.00,22.40,99238804,24.70,22.00,B,SET100,0.40,1.82
7,MCS,2025-01-08,2025-01-16,6.45,6.35,8760160,6.60,6.30,B,SET,-0.10,-1.55
8,NER,2025-01-15,2025-01-16,4.80,4.90,49086766,4.90,4.48,B,SET,0.10,2.08
9,SENA,2025-01-09,2025-01-16,2.20,2.20,13598960,2.24,2.16,B,SET,0.00,0.00


In [16]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [20]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-14,2025-01-16,5.05,5.10,"23,196,304",5.15,5.05,0.99%,B
1,AIMIRT,2025-01-10,2025-01-16,10.60,10.50,"1,276,096",10.60,10.40,-0.94%,B
2,CPNREIT,2025-01-16,2025-01-16,12.20,12.10,"19,887,990",12.30,11.80,-0.82%,B
3,DIF,2025-01-16,2025-01-16,8.20,8.25,"43,884,384",8.45,8.20,0.61%,B
4,GVREIT,2025-01-10,2025-01-16,6.55,6.65,"6,604,800",6.65,6.45,1.53%,B
5,JMART,2025-01-16,2025-01-16,11.50,11.20,"31,715,008",11.70,11.10,-2.61%,B
6,KCE,2025-01-15,2025-01-16,22.00,22.40,"99,238,804",24.70,22.00,1.82%,B
7,MCS,2025-01-08,2025-01-16,6.45,6.35,"8,760,160",6.60,6.30,-1.55%,B
8,NER,2025-01-15,2025-01-16,4.80,4.90,"49,086,766",4.90,4.48,2.08%,B
9,SENA,2025-01-09,2025-01-16,2.20,2.20,"13,598,960",2.24,2.16,0.00%,B


In [22]:
file_name = "daily-sales.csv"
data_file = data_path + '/' + file_name
output_file = csv_path + '/' + file_name
one_file = one_path + '/' + file_name
osd_file = osd_path + '/' + file_name
print(data_file)

../data/daily-sales.csv


In [24]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [26]:
sales = df[cols]
sales.shape

(20, 10)

In [28]:
file_name = "Price-Today.csv"
input_file = data_path + '/' + file_name
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [30]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [32]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-14,2025-01-16,5.05,5.10,23196304,5.15,5.05,0.99,B,5.10,0.00,3062050,2025-01-17
1,AIMIRT,2025-01-10,2025-01-16,10.60,10.50,1276096,10.60,10.40,-0.94,B,10.50,0.00,135527,2025-01-17
2,CPNREIT,2025-01-16,2025-01-16,12.20,12.10,19887990,12.30,11.80,-0.82,B,12.20,0.10,1077970,2025-01-17
3,DIF,2025-01-16,2025-01-16,8.20,8.25,43884384,8.45,8.20,0.61,B,8.30,0.05,4880270,2025-01-17
4,GVREIT,2025-01-10,2025-01-16,6.55,6.65,6604800,6.65,6.45,1.53,B,6.65,0.00,148255,2025-01-17
5,JMART,2025-01-16,2025-01-16,11.50,11.20,31715008,11.70,11.10,-2.61,B,10.90,-0.30,2506075,2025-01-17
6,KCE,2025-01-15,2025-01-16,22.00,22.40,99238804,24.70,22.00,1.82,B,21.70,-0.70,6559285,2025-01-17
7,MCS,2025-01-08,2025-01-16,6.45,6.35,8760160,6.60,6.30,-1.55,B,6.35,0.00,70500,2025-01-17
8,NER,2025-01-15,2025-01-16,4.80,4.90,49086766,4.90,4.48,2.08,B,4.84,-0.06,8175729,2025-01-17
9,SENA,2025-01-09,2025-01-16,2.20,2.20,13598960,2.24,2.16,0.00,B,2.20,0.00,109622,2025-01-17


In [34]:
file_name = "daily-sales-prices.csv"
data_file = data_path + '/' + file_name
output_file = csv_path + '/' + file_name
one_file = one_path + '/' + file_name
osd_file = osd_path + '/' + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [37]:
# Get the current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

Current Directory: C:\Users\PC1\OneDrive\A5\Daily


In [39]:
# Derive the base directory (to_dir) by removing the last folder ('Daily')
to_dir = os.path.dirname(current_dir)
print(f"Base Directory (to_dir): {to_dir}")

Base Directory (to_dir): C:\Users\PC1\OneDrive\A5


In [41]:
os.chdir(to_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [43]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-14 2025-01-17   5.05   5.10   0.99%  23,196,304   5.15   5.05 B Update
AIMIRT   2025-01-10 2025-01-17  10.60  10.50  -0.94%   1,276,096  10.60  10.40 B Update
AWC      2025-01-17 2025-01-17   3.18   3.14  -1.26% 2,138,788,616   3.58   3.08 S Insert
BCH      2025-01-16 2025-01-17  14.80  14.50  -2.03% 115,687,072  15.70  14.20 S Update
CPNREIT  2025-01-17 2025-01-17  12.10  12.20   0.83%  19,887,990  12.30  11.80 B Insert
DIF      2025-01-16 2025-01-17   8.20   8.30   1.22%  43,884,384   8.45   8.20 B Update
GVREIT   2025-01-10 2025-01-17   6.55   6.65   1.53%   6,604,800   6.65   6.45 B Update
JMART    2025-01-16 2025-01-17  11.50  10.90  -5.22%  31,715,008  11.70  10.90 B Update
JMT      2025-01-16 2025-01-17  15.50  14.40   -7.1% 271,009,584  16.20  14.40 S Update
KCE      2025-01-17 2025-01-17

In [45]:
os.chdir(current_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [47]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-15' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [49]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [51]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-14,2025-01-15,5.05,5.10,"11,598,152",5.15,5.05,B,SET,0.05,0.99%
1,AIMIRT,2025-01-10,2025-01-15,10.60,10.50,"638,048",10.60,10.40,B,SET,-0.10,-0.94%
2,CPNREIT,2025-01-15,2025-01-15,12.10,12.20,"19,887,990",12.30,11.80,B,SET,0.10,0.83%
3,DIF,2025-01-10,2025-01-15,8.45,8.20,"43,884,384",8.45,8.20,B,SET,-0.25,-2.96%
4,GVREIT,2025-01-10,2025-01-15,6.55,6.65,"3,302,400",6.65,6.45,B,SET,0.10,1.53%
5,JMART,2025-01-15,2025-01-15,11.10,11.50,"31,715,008",11.70,11.10,B,SET50,0.40,3.60%
6,KCE,2025-01-15,2025-01-15,22.00,22.40,"49,619,402",24.70,22.00,B,SET100,0.40,1.82%
7,MCS,2025-01-08,2025-01-15,6.45,6.35,"4,380,080",6.60,6.30,B,SET,-0.10,-1.55%
8,NER,2025-01-15,2025-01-15,4.80,4.86,"24,543,383",4.86,4.48,B,SET,0.06,1.25%
9,SENA,2025-01-09,2025-01-15,2.20,2.20,"6,799,480",2.24,2.16,B,SET,0.00,0.00%


In [53]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [55]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,TVO,2025-01-02,2025-01-15,22.90,21.50,"22,402,704",23.20,21.40,S,SET,-1.40,-6.11%


In [57]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [59]:
conlite.close()